<a href="https://colab.research.google.com/github/tinkvu/thesis-english/blob/main/AnyLanguagetoEnglishTTS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
# Install required packages
!pip install groq gtts ffmpeg-python pydub

In [19]:
import os
from groq import Groq
from gtts import gTTS
from pydub import AudioSegment
from IPython.display import Audio, display

# Initialize the Groq client
API_KEY = ""
client = Groq(api_key=API_KEY)

# Function to transcribe audio using Groq Whisper API
def transcribe_audio(file_path, model="whisper-large-v3"):
    with open(file_path, "rb") as file:
        transcription = client.audio.transcriptions.create(
            file=(os.path.basename(file_path), file.read()),
            model=model,
            response_format="verbose_json",
        )
    return transcription

# Function to play audio in Colab using gTTS
def play_audio_with_gtts(text, output_file="output_audio.mp3"):
    tts = gTTS(text)
    tts.save(output_file)
    return output_file

# Function to translate text using Groq Chat Completion model
def translate_to_english(text):
    completion = client.chat.completions.create(
        model="llama-3.2-90b-text-preview",
        messages=[
            {
                "role": "system",
                "content": f"translate this to english: {text}\nOnly output the translation.\nExample:\nInput: \"translate this to english: Oye, como estas?\"\nOutput: \"Hey, how are you?\""
            },
                   ],
        temperature=1,
        max_tokens=1024,
        top_p=1,
        stream=True,
        stop=None,
    )

    # Collect the translation response
    translation = ""
    for chunk in completion:
        translation += chunk.choices[0].delta.content or ""

    return translation.strip()

# Step 1: Upload an audio file
from google.colab import files

uploaded = files.upload()  # Upload your audio file
if uploaded:
    audio_file = list(uploaded.keys())[0]
    print(f"Uploaded file: {audio_file}")

    # Convert to a supported format if needed (e.g., .mp3 or .wav)
    if not audio_file.endswith((".mp3", ".wav")):
        print("Converting audio file to .wav format...")
        sound = AudioSegment.from_file(audio_file)
        audio_file = "converted_audio.wav"
        sound.export(audio_file, format="wav")
        print("Conversion complete.")

    # Step 2: Transcribe the audio
    print("Transcribing audio...")
    try:
        transcription = transcribe_audio(audio_file)

        # Access transcription text and language attributes directly
        transcription_text = transcription.text  # Accessing the 'text' attribute directly
        transcription_language = transcription.language  # Accessing the 'language' attribute directly

        print("\nTranscription Results:")
        print(transcription_text)

        # Check if the transcription is not in English
        if transcription_language != "en":
            print("\nTranscription is not in English. Translating to English...")

            # Translate the transcription to English using Groq chat completions
            try:
                translated_text = translate_to_english(transcription_text)
                print("\nTranslated Text:")
                print(translated_text)

                # Play the translated text as speech (English translation)
                print("\nPlaying the translation as speech:")
                translated_audio_output = play_audio_with_gtts(translated_text, "translated_output_audio.mp3")
                display(Audio(translated_audio_output, autoplay=True))
            except Exception as e:
                print(f"An error occurred during translation: {e}")
        else:
            # If the transcription is already in English, play it directly
            print("\nTranscription is already in English. Playing the transcription as speech:")
            audio_output = play_audio_with_gtts(transcription_text)
            display(Audio(audio_output, autoplay=True))
    except Exception as e:
        print(f"An error occurred during transcription: {e}")


Saving ttsMP3.com_VoiceText_2024-11-18_14-51-3.mp3 to ttsMP3.com_VoiceText_2024-11-18_14-51-3.mp3
Uploaded file: ttsMP3.com_VoiceText_2024-11-18_14-51-3.mp3
Transcribing audio...

Transcription Results:
 ¿Dónde puedo coger el autobús al centro de la ciudad?

Transcription is not in English. Translating to English...

Translated Text:
Where can I catch the bus to the city center?

Playing the translation as speech:
